# GridSearch

Perform a grid search given a target optimizer, a target loss, a target model/dataset and a target L-norm.

**1. Setting Colab enviroments**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import subprocess
import sys
import os
from tqdm import tqdm
os.getcwd()

'/content'

In [3]:
cd /content/gdrive/My Drive/Colab Notebooks/Adversarial/AdvAttack/scripts

/content/gdrive/My Drive/Colab Notebooks/Adversarial/AdvAttack/scripts


In [4]:
print(os.getcwd())
print(os.listdir())

/content/gdrive/My Drive/Colab Notebooks/Adversarial/AdvAttack/scripts
['test.py', 'evaluation.py', 'Grid_evaluation.ipynb']


**2. Define params**

In [0]:
OPTIM = 'classic'
LOSS = '-1'
DATA = 'cifar10'
EPSILON = '0.01'
MAXIMISE = '0'
N_EXAMPLE = '100'
TQDM = '1'


MAX_BATCH_SIZE = 4000

params_list = [ 
               {'n_gradient':  500},
               {'n_gradient':  1000},
               {'n_gradient':  2000},
               {'n_gradient':  4000},
               {'n_gradient':  8000},
               {'n_gradient':  16000},
               ]

**3. Run the evaluation.py script**

In [0]:
for params in tqdm(params_list):
    exe_list = ['python', 'evaluation.py', 
                '--optimizer', OPTIM, 
                '--data', DATA, 
                '--epsilon', EPSILON, 
                '--L_type', LOSS, 
                '--alpha', '0.2', 
                '--maximise', MAXIMISE, 
                '--n_example', N_EXAMPLE,
                '--data_batch_size', N_EXAMPLE]

    if params['n_gradient'] > MAX_BATCH_SIZE:
        exe_list.append('--batch_size')
        exe_list.append(str(MAX_BATCH_SIZE))
    
    for k in params:
        exe_list.append('--{}'.format(k))
        exe_list.append(str(params[k]))
    
    print(exe_list)
    out = subprocess.run(exe_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)

  0%|          | 0/6 [00:00<?, ?it/s]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '0', '--n_example', '100', '--data_batch_size', '100', '--n_gradient', '500']


 17%|█▋        | 1/6 [13:40<1:08:24, 820.93s/it]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '0', '--n_example', '100', '--data_batch_size', '100', '--n_gradient', '1000']


 33%|███▎      | 2/6 [32:29<1:00:53, 913.33s/it]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '0', '--n_example', '100', '--data_batch_size', '100', '--n_gradient', '2000']


 50%|█████     | 3/6 [57:01<54:02, 1080.97s/it] 

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '0', '--n_example', '100', '--data_batch_size', '100', '--n_gradient', '4000']


 67%|██████▋   | 4/6 [1:31:31<45:55, 1377.64s/it]

['python', 'evaluation.py', '--optimizer', 'classic', '--data', 'cifar10', '--epsilon', '0.01', '--L_type', '-1', '--alpha', '0.2', '--maximise', '0', '--n_example', '100', '--data_batch_size', '100', '--batch_size', '4000', '--n_gradient', '8000']


**4. See results**


In [0]:
import json
import pandas as pd
path = '../logs/evaluation_logs'

r = []
for file_name in os.listdir(path):
    with open('{}/{}'.format(path, file_name), 'r') as file:
        logs = json.load(file)
        if logs['All_config_args']['epsilon'] ==  float(EPSILON) and logs['Dataset'] ==  DATA and logs['L norm'] ==  int(LOSS) and logs['Optimizer'] ==  OPTIM and logs['Target attack'] == 0:
            print(logs)
            a, n, v = logs['All_config_args']['alpha'], logs['All_config_args']['n_gradient'], logs['All_config_args']['v']
            s, t = logs['Results']['Success'], logs['Results']['Mean time']
            r.append([s, t, n, a, v])


df = pd.DataFrame(r)
df.columns = ['Succes Rate', 'Time per attack', '# Gradient Estimation', 'Alpha', 'Gaussian Smoothing']

res_path = '../results/{}_{}_untarget_{}.csv'.format(OPTIM, DATA, str(EPSILON)[-2:])
print('Results saved at {}'.format(res_path))
df.to_csv(res_path)
df.sort_values(by=['Succes Rate', 'Time per attack'], axis=0, ascending=False)